# The purpose of this notebook is to create datasets consisting of bitcoin prices together with lagged tweets

In [1]:
#imports
import pandas as pd 
from datetime import datetime
from datetime import timedelta
import os

### Load BTC data with dates

In [2]:
btc_data = pd.read_csv("./../datasets/general/BTCDATAwithdate.csv") 

In [3]:
btc_data.head()

,Unnamed: 0,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,timestamp
0,3500000,6936.26,1.001363,6944.466619,6935.011702,2018-08-31 11:04:00+00:00
1,3500001,6944.90,10.379585,72022.107341,6938.823172,2018-08-31 11:05:00+00:00
2,3500002,6949.89,1.264825,8789.841563,6949.454444,2018-08-31 11:06:00+00:00
3,3500003,6950.61,1.948446,13550.262119,6954.393879,2018-08-31 11:07:00+00:00
4,3500004,6949.51,5.503697,38235.502385,6947.240108,2018-08-31 11:08:00+00:00


In [4]:
#Change timestamp to data in bitcoin data
# btc_data['timestamp'] = [datetime.fromtimestamp(x for x in btc_data['Timestamp']]
btc_data['timestamp'] = pd.to_datetime(btc_data['timestamp'], utc = True)  

## According to the analyser used to create polarity scores, load the file

In [5]:
#this can either be TextBlob or vader
analyser = "vader"
# polarity_medium = "vader"

#get tweets
tweet_data = pd.read_csv("./../datasets/general/with_sentiment/"+analyser+"/tweets.csv")

C:\Users\franc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,2,3,4,5,6,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
tweet_data.head()

,Unnamed: 0,Unnamed: 0.1,id,user,fullname,url,timestamp,replies,likes,retweets,text,compound
0,0,0,1132977055300300800,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 11:49:14+00,0.0,0.0,0.0,È appena uscito un nuovo video! LES CRYPTOMONN...,0.0000
1,1,1,1132977073402736640,bitcointe,Bitcointe,NaN,2019-05-27 11:49:18+00,0.0,0.0,0.0,Cardano: Digitize Currencies; EOS https://t.co...,-0.1027
2,2,2,1132977023893139456,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:06+00,0.0,2.0,1.0,Another Test tweet that wasn't caught in the s...,0.0000
3,3,3,1132977089089556481,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:22+00,0.0,0.0,0.0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,0.0000
4,4,4,1132977092340191232,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:23+00,0.0,0.0,0.0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,0.3612


### Create lag

In [8]:
#choose lag
lag = 1
#this can be hours or days
granularity = "hours"

In [9]:
#copy tweet data and move by lag
tweet_data_copy = tweet_data
tweet_data_copy['timestamp'] = pd.to_datetime(tweet_data_copy['timestamp'], utc = True)  
tweet_data_copy['timestamp'] = tweet_data_copy['timestamp'] - timedelta(hours=lag)

In [10]:
tweet_data_copy.head()

,Unnamed: 0,Unnamed: 0.1,id,user,fullname,url,timestamp,replies,likes,retweets,text,compound
0,0,0,1132977055300300800,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 10:49:14+00:00,0.0,0.0,0.0,È appena uscito un nuovo video! LES CRYPTOMONN...,0.0000
1,1,1,1132977073402736640,bitcointe,Bitcointe,NaN,2019-05-27 10:49:18+00:00,0.0,0.0,0.0,Cardano: Digitize Currencies; EOS https://t.co...,-0.1027
2,2,2,1132977023893139456,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 10:49:06+00:00,0.0,2.0,1.0,Another Test tweet that wasn't caught in the s...,0.0000
3,3,3,1132977089089556481,DetroitCrypto,J. Scardina,NaN,2019-05-27 10:49:22+00:00,0.0,0.0,0.0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,0.0000
4,4,4,1132977092340191232,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 10:49:23+00:00,0.0,0.0,0.0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,0.3612


In [11]:
#Round timestamps to nearest minute
tweet_data_copy['timestamp'] = tweet_data_copy['timestamp'].apply(lambda dt: dt.round('min'))

In [12]:
tweet_data_copy.head()

,Unnamed: 0,Unnamed: 0.1,id,user,fullname,url,timestamp,replies,likes,retweets,text,compound
0,0,0,1132977055300300800,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 10:49:00+00:00,0.0,0.0,0.0,È appena uscito un nuovo video! LES CRYPTOMONN...,0.0000
1,1,1,1132977073402736640,bitcointe,Bitcointe,NaN,2019-05-27 10:49:00+00:00,0.0,0.0,0.0,Cardano: Digitize Currencies; EOS https://t.co...,-0.1027
2,2,2,1132977023893139456,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 10:49:00+00:00,0.0,2.0,1.0,Another Test tweet that wasn't caught in the s...,0.0000
3,3,3,1132977089089556481,DetroitCrypto,J. Scardina,NaN,2019-05-27 10:49:00+00:00,0.0,0.0,0.0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,0.0000
4,4,4,1132977092340191232,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 10:49:00+00:00,0.0,0.0,0.0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,0.3612


In [13]:
#convert btc data to UTC
btc_data['timestamp'] = pd.to_datetime(btc_data['timestamp'], utc = True)  

In [14]:
#concatinate btc prices and tweets on timestamp
final_data = btc_data.merge(tweet_data_copy, on="timestamp")

In [15]:
final_data.head()

,Unnamed: 0_x,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,timestamp,Unnamed: 0_y,Unnamed: 0.1,id,user,fullname,url,replies,likes,retweets,text,compound
0,3500000,6936.26,1.001363,6944.466619,6935.011702,2018-08-31 11:04:00+00:00,556350,556350,1035528576672530433,bitcoin_whales,Whales NEWS,/bitcoin_whales/status/1035528576672530433,2.0,6.0,0.0,#BTC #CME Futures expiring today 8:00 AM PST. ...,0.000
1,3500001,6944.90,10.379585,72022.107341,6938.823172,2018-08-31 11:05:00+00:00,2628038,2628038,1035528763457433600,DolarTrue_,DolarTrue,/DolarTrue_/status/1035528763457433600,0.0,1.0,1.0,#DolarTrue BTC\n31/08/2018 10:04 AM\nBTC Venta...,0.000
2,3500001,6944.90,10.379585,72022.107341,6938.823172,2018-08-31 11:05:00+00:00,2628039,2628039,1035528970626682880,KoinKnight,KoinKnight,/KoinKnight/status/1035528970626682880,0.0,0.0,0.0,Price Alert: #NGC is going down.#NGC price has...,0.296
3,3500007,6939.11,6.495773,45080.950674,6940.044005,2018-08-31 11:11:00+00:00,2628035,2628035,1035530457545408512,tr_ticker,Tr Ticker Bot,/tr_ticker/status/1035530457545408512,0.0,0.0,0.0,#Doviz\n-------------------\n#USD : 6.6067\n#E...,0.000
4,3500007,6939.11,6.495773,45080.950674,6940.044005,2018-08-31 11:11:00+00:00,2628036,2628036,1035530316989882368,DTradingAcademy,Day Trading Academy,/DTradingAcademy/status/1035530316989882368,0.0,1.0,0.0,"Cryptocurrency Bitcoin -0.5% at $6,945.00 in F...",0.000


In [16]:
#drop rows where close is null
final_data = final_data.dropna(axis=0, subset=['Close'])

In [17]:
final_data.head()

,Unnamed: 0_x,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,timestamp,Unnamed: 0_y,Unnamed: 0.1,id,user,fullname,url,replies,likes,retweets,text,compound
0,3500000,6936.26,1.001363,6944.466619,6935.011702,2018-08-31 11:04:00+00:00,556350,556350,1035528576672530433,bitcoin_whales,Whales NEWS,/bitcoin_whales/status/1035528576672530433,2.0,6.0,0.0,#BTC #CME Futures expiring today 8:00 AM PST. ...,0.000
1,3500001,6944.90,10.379585,72022.107341,6938.823172,2018-08-31 11:05:00+00:00,2628038,2628038,1035528763457433600,DolarTrue_,DolarTrue,/DolarTrue_/status/1035528763457433600,0.0,1.0,1.0,#DolarTrue BTC\n31/08/2018 10:04 AM\nBTC Venta...,0.000
2,3500001,6944.90,10.379585,72022.107341,6938.823172,2018-08-31 11:05:00+00:00,2628039,2628039,1035528970626682880,KoinKnight,KoinKnight,/KoinKnight/status/1035528970626682880,0.0,0.0,0.0,Price Alert: #NGC is going down.#NGC price has...,0.296
3,3500007,6939.11,6.495773,45080.950674,6940.044005,2018-08-31 11:11:00+00:00,2628035,2628035,1035530457545408512,tr_ticker,Tr Ticker Bot,/tr_ticker/status/1035530457545408512,0.0,0.0,0.0,#Doviz\n-------------------\n#USD : 6.6067\n#E...,0.000
4,3500007,6939.11,6.495773,45080.950674,6940.044005,2018-08-31 11:11:00+00:00,2628036,2628036,1035530316989882368,DTradingAcademy,Day Trading Academy,/DTradingAcademy/status/1035530316989882368,0.0,1.0,0.0,"Cryptocurrency Bitcoin -0.5% at $6,945.00 in F...",0.000


In [18]:
#remove unwanted column
final_data = final_data.drop(["Unnamed: 0_x", "Unnamed: 0_y"], axis=1)

In [19]:
final_data.head()

,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,timestamp,Unnamed: 0.1,id,user,fullname,url,replies,likes,retweets,text,compound
0,6936.26,1.001363,6944.466619,6935.011702,2018-08-31 11:04:00+00:00,556350,1035528576672530433,bitcoin_whales,Whales NEWS,/bitcoin_whales/status/1035528576672530433,2.0,6.0,0.0,#BTC #CME Futures expiring today 8:00 AM PST. ...,0.000
1,6944.90,10.379585,72022.107341,6938.823172,2018-08-31 11:05:00+00:00,2628038,1035528763457433600,DolarTrue_,DolarTrue,/DolarTrue_/status/1035528763457433600,0.0,1.0,1.0,#DolarTrue BTC\n31/08/2018 10:04 AM\nBTC Venta...,0.000
2,6944.90,10.379585,72022.107341,6938.823172,2018-08-31 11:05:00+00:00,2628039,1035528970626682880,KoinKnight,KoinKnight,/KoinKnight/status/1035528970626682880,0.0,0.0,0.0,Price Alert: #NGC is going down.#NGC price has...,0.296
3,6939.11,6.495773,45080.950674,6940.044005,2018-08-31 11:11:00+00:00,2628035,1035530457545408512,tr_ticker,Tr Ticker Bot,/tr_ticker/status/1035530457545408512,0.0,0.0,0.0,#Doviz\n-------------------\n#USD : 6.6067\n#E...,0.000
4,6939.11,6.495773,45080.950674,6940.044005,2018-08-31 11:11:00+00:00,2628036,1035530316989882368,DTradingAcademy,Day Trading Academy,/DTradingAcademy/status/1035530316989882368,0.0,1.0,0.0,"Cryptocurrency Bitcoin -0.5% at $6,945.00 in F...",0.000


### Save to dataset file

In [20]:
folder = './../datasets/tweets_and_prices/'+analyser

if not os.path.exists(folder):
    os.makedirs(folder)

In [21]:
filename = folder+"/final_data_lag_"+granularity+"_"+str(lag)+".csv" if (lag > 0) else folder+"/final_data_no_lag.csv"
final_data.to_csv(filename)